In [169]:
import json
%pip install -U --quiet langchain_openai langsmith langgraph langchain numexpr langchainhub


Note: you may need to restart the kernel to use updated packages.


In [170]:
%reload_ext autoreload
%autoreload 2

In [178]:
from langchain import hub

planer_prompt = hub.pull("wfh/llm-compiler")
planer_prompt.pretty_print()

print("========== replan =====")

replanner_prompt = planer_prompt.partial(
        replan=' - You are given "Previous Plan" which is the plan that the previous agent created along with the execution results '
        "(given as Observation) of each plan and a general thought (given as Thought) about the executed results."
        'You MUST use these information to create the next plan under "Current Plan".\n'
        ' - When starting the Current Plan, you should start with "Thought" that outlines the strategy for the next plan.\n'
        " - In the Current Plan, you should NEVER repeat the actions that are already executed in the Previous Plan.\n"
        " - You must continue the task index from the end of the previous one. Do not repeat task indices.",
        num_tools=2,
        tool_descriptions="blalbba"
    )

replanner_prompt.pretty_print()
print("========== joiner =====")

joiner_prompt = hub.pull("wfh/llm-compiler-joiner")
joiner_prompt.pretty_print()






================================ System Message ================================

Given a user query, create a plan to solve it with the utmost parallelizability. Each plan should comprise an action from the following {num_tools} types:
{tool_descriptions}
{num_tools}. join(): Collects and combines results from prior actions.

 - An LLM agent is called upon invoking join() to either finalize the user query or wait until the plans are executed.
 - join should always be the last action in the plan, and will be called in two scenarios:
   (a) if the answer can be determined by gathering the outputs from tasks to generate the final response.
   (b) if the answer cannot be determined in the planning phase before you execute the plans. Guidelines:
 - Each action described above contains input/output types and description.
    - You must strictly adhere to the input and output types for each action.
    - The action descriptions contain the guidelines. You MUST strictly follow those guideline

In [172]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.callbacks import CallbackManagerForChainRun, StdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults
from langchain.chains import LLMMathChain
from langchain_community.llms import OpenAI

chat_model = ChatOpenAI(temperature = 0, model_name ="gpt-4-turbo")
# chat_model = ChatOpenAI(temperature = 0, openai_api_base="http://192.168.0.134:8000/v1", model_name="TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ")
# chat_model = ChatOpenAI(temperature = 0, openai_api_base="http://192.168.0.134:8000/v1", model_name="/root/models/Meta/Meta-Llama-3-70B-Instruct-AWQ/")
# chat_model.invoke("hello")

class CalculateSchema(BaseModel):
    question: str = Field(description="a math expression. ")
 

# @tool("calculate", args_schema=CalculateSchema)
# def calculate(param: CalculateSchema) -> float:
#     """
#     Useful for when you need to answer questions about math.
#     """
#     handler = StdOutCallbackHandler()
#     chain = LLMMathChain.from_llm(chat_model, verbose=True, callbacks=[handler])
#     return chain.invoke(param.question)


@tool("calculate", args_schema=CalculateSchema, )
def calculate(question: str) -> float:
    """
    Useful for evaluating a math expression.
    """
    handler = StdOutCallbackHandler()
    chain = LLMMathChain.from_llm(chat_model, verbose=True, callbacks=[handler])
    return chain.invoke(question)

    

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

tools = [
    
    tavily_tool,
    calculate,
]


# test calculator
# calculate.invoke("What's result of 1 + 3?")

# test search
# tavily_tool.invoke("What's the GDP of China in year 2023?")


In [173]:
from langchain_core.tools import render_text_description
import json


tool_description = ""

for i, tool in enumerate(tools):
    tool_description += f"{i+1}. ({tool.name}):  {tool.description},  arguments JSON schema: {tool.args} \n"

# tool_description = render_text_description(tools)

print(f"tool_description: \n{tool_description}")

tool_description: 
1. (tavily_search_results_json):  A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.,  arguments JSON schema: {'query': {'title': 'Query', 'description': 'search query to look up', 'type': 'string'}} 
2. (calculate):  calculate(question: str) -> float - Useful for evaluating a math expression.,  arguments JSON schema: {'question': {'title': 'Question', 'description': 'a math expression. ', 'type': 'string'}} 



In [177]:
from langchain_core.messages import HumanMessage
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import MessagesPlaceholder
from tqdm import tqdm


planer_prompt = ChatPromptTemplate.from_messages(
    messages=[
        ("user", r"""Given a user query, create a plan to solve it with the utmost parallelization. Each plan should comprise an action from the following {num_tools} types:
    {tool_descriptions}
    {num_tools}. join(): Collects and combines results from prior actions. No arguments needed.
    
     - An LLM agent is called upon invoking join() to either finalize the user query or wait until the plans are executed.
     - join should always be the last action in the plan, and will be called in two scenarios:
       (a) if the answer can be determined by gathering the outputs from tasks to generate the final response.
       (b) if the answer cannot be determined in the planning phase before you execute the plans. Guidelines:
     - Each action described above contains input/output types and description.
        - You must strictly adhere to the input and output types for each action.
        - The action descriptions contain the guidelines. You MUST strictly follow those guidelines when you use the actions.
     - Each action in the plan should strictly be one of the above types.
     - Each action MUST have a unique ID, which is strictly increasing.
     - Input to the action is formatted as JSON blob with 'name' and 'arguments' keys.
     - If inputs for actions are outputs from preceding actions,  always use the format $id to denote the ID of the previous action whose output will be used as the input.
     - Always call join as the last action in the plan. Say '<END_OF_PLAN>' after you call join in a new line.
     - Ensure the plan maximizes parallelization.
     - Only use the provided action types. If a query cannot be addressed using these, invoke the join action for the next steps.
     - Never introduce new actions other than the ones provided.
     
    Remember, ONLY respond with the task list in the following format:
    ID. JSON blob of action input
    
    {replan_instruction}
    
    Question: {question}
    
    {context}
    """ )
    ]
)


# chat_model.bind(stop=["<END_OF_PLAN>"])
planer = planer_prompt | chat_model


for question in  tqdm([
    "What's today's temperature in Shanghai raised to 2nd power?",
    "How's the cheapest Apple Macbook compared to cheapest Lenovo Thinkpad in terms of CPU performance in the year of 2023?",
    "What's the total number of the Olympic modals won by the first three countries by populations in 2000",
]):
    print(planer.invoke(input={
        "question": question,
        "num_tools": len(tools) + 1,
        "tool_descriptions": tool_description,
        "replan_instruction": "",
        "context": ""
    }, config={}))
    
    # print("====")
    # 
    # for msg in planer_prompt.invoke({
    #     "messages": [HumanMessage (question)],
    #     "num_tools": len(tools) + 1,
    #     "tool_descriptions": tool_description
    # }).to_messages():
    #     print(msg.content)
    #     print("====")


KeyboardInterrupt: 

In [ ]:

# content='1. {"name": "tavily_search_results_json", "arguments": {"query": "today\'s temperature in Shanghai"}}\n2. {"name": "calculate", "arguments": {"question": "$1^2"}}\n3. {"name": "join"}\n<END_OF_PLAN>' response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 533, 'total_tokens': 591}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_294de9593d', 'finish_reason': 'stop', 'logprobs': None} id='run-7c0e96ba-0d4d-43a7-87a4-c27b5d003d37-0'





replan_prompt = ChatPromptTemplate.from_messages(messages=[
    ("user", r"""
Solve a question answering task. Here are some guidelines:
 - In the Assistant Scratchpad, you will be given results of a plan you have executed to answer the user's question.
 - Thought needs to reason about the question based on the Observations in 1-2 sentences.
 - Ignore irrelevant action results.
 - If the required information is present, give a concise but complete and helpful answer to the user's question.
 - If you are unable to give a satisfactory finishing answer, replan to get the required information. Respond in the following format:

Thought: <reason about the task results and whether you have sufficient information to answer the question>
Action: <action to take>
Available actions:
 (1) Finish(the final answer to return to the user): returns the answer and finishes the task.
 (2) Replan(the reasoning and other information that will help you plan again. Can be a line of any length): instructs why we must replan
 
 
 
 
 
 Using the above previous actions, decide whether to replan or finish. If all the required information is present. You may finish. If you have made many attempts to find the information without success, admit so and respond with whatever information you have gathered so the user can work well with you.
"""),
    
    
])